# Preprocess the data

## Rigid registration and skull extraction

In [3]:
import os
from os.path import expanduser as eu
from headctools.registration import register_file_folder
from headctools.preprocessing import ct_skull

In [ ]:
imgs = eu('~/Code/datasets/cq500/converted/selected/')
fixed_img = os.path.join(imgs, 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz')
# Registered images will be saved in a sub folder with the name of the fixed image

params = {
    'reg_atlas_path': fixed_img
}

ct_skull(imgs, params = params)

## Mesh atlas construction

### Decimate meshes

In [6]:
import vedo

print('Decimating meshes...')
decimate_percent = 0.01

meshes_folder = eu('~/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/test2/preprocessed_ct_to_skull')
print( f'  Input folder: {meshes_folder}')
meshes_paths = [os.path.join(meshes_folder, f) for f in os.listdir(meshes_folder) if f.endswith('.stl') and 'decimated' not in f]
print(f'  Found {len(meshes_paths)} meshes')
for i, mesh in enumerate(meshes_paths):
    print(f'    [{i+1}/{len(meshes_paths)}] Input mesh: {os.path.split(mesh)[1]}')
    v_mesh = vedo.Mesh(mesh).decimate(decimate_percent)
    decimated_path = mesh.replace('.stl', f'_decimated_{int(decimate_percent*100)}perc_{v_mesh.points().shape[0]}points.stl')
    v_mesh.write(decimated_path)
    print(f'      Decimated mesh saved in {decimated_path}')


Decimating meshes...
  Input folder: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/test2/preprocessed_ct_to_skull
  Found 4 meshes
    [0/4] Input mesh: CQ500-CT-390_CT 5mm POST CONTRAST.stl
      Decimated mesh saved in /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/test2/preprocessed_ct_to_skull/CQ500-CT-390_CT 5mm POST CONTRAST_decimated_1perc_6063points.stl
    [1/4] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN.stl
      Decimated mesh saved in /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/test2/preprocessed_ct_to_skull/CQ500-CT-312_CT PRE CONTRAST THIN_decimated_1perc_7345points.stl
    [2/4] Input mesh: CQ500-CT-405_CT Thin Plain.stl
      Decimated mesh saved in /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/test2/preprocessed_ct_to_skull/CQ500-CT-405_CT Thin Plain_decimated_1perc_6907points.stl
    [3/4] Input mesh: CQ500-CT-216_CT 0.625